# 1.TensorFlow用のQuantizationツールをインストール

In [ ]:
!pip install intel-quantization

# 2.量子化するモデルをダウンロード

In [ ]:
!wget https://storage.googleapis.com/intel-optimized-tensorflow/models/resnet50_fp32_pretrained_model.pb api/models/resnet50/resnet50_fp32_pretrained_model.pb

# 3.推論スクリプトのダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/hiouchiy/IntelAI/master/tensorflow_quantization/infer_script.py

推論スクリプト（infer_script.py）について。

IntelAIのGithubレポジトリには、いわゆる公式ベンチマークツールがあるのですが、今回はあえてそちらを使わずに（というかより実践的な状況を想定して）独自の推論スクリプトを用意しました。公式ツールよりも実装が緩いため、若干性能は劣りますが現実感を重要視しておりますので何卒ご容赦ください。

スクリプトパラメータの説明
- --input_graph・・・モデルファイルのパス
- --dataset_dir・・・画像データフォルダのパス
- --num_images・・・推論する画像枚数（この枚数を上記画像フォルダからランダムに選んで推論します。）
- --openvino・・・OpenVINOの推論エンジン上で推論する場合はこちらにTrueをセット下さい。当然ながらモデルファイルはIRをして下さい。

# 4.FP32モデルの性能確認

ここでは、推論スクリプトの実行確認を兼ねて、量子化前のFP32のモデルの性能を確認してみます。

In [ ]:
!numactl -l python3 infer_script.py --input_graph "api/models/resnet50/resnet50_fp32_pretrained_model.pb" --dataset_dir /imagenet/images --num_images 50

# 5.モデルの入力Op、および、出力Opの名称を確認
量子化を行うためには、モデルの入力および出力のOps名が必要です。つまり、どのOpsからどのOpsまでを量子化対象とするのか、決める必要があるからです。ユーティリティツールとして"api/tools/summarize_graph.py"が用意されていますので、下記のようにご使用下さい。

In [ ]:
!python3 api/tools/summarize_graph.py --in_graph=api/models/resnet50/resnet50_fp32_pretrained_model.pb --input_binary

# 6.量子化実行

量子化を実行する方法はいくつかあるのですが、ここでは恐らく最も汎用的な"api/examples/quantize_cmd.py"を使った方法をご紹介します。

In [ ]:
!python api/examples/quantize_cmd.py --input_graph api/models/resnet50/resnet50_fp32_pretrained_model.pb --output_graph  api/models/resnet50/resnet50_int8_pretrained_model.pb --callback "python3 ./infer_tf.py --input_graph {} --dataset_dir /imagenet/images --num_images 1000" --inputs 'input' --outputs 'predict' --per_channel False --excluded_ops '' --excluded_nodes ''

実行中に各パラメータの意味を確認しましょう。

- --input_graph api/models/resnet50/resnet50_fp32_pretrained_model.pb・・・元のFP32のモデルファイルのパス
- --output_graph  api/models/resnet50/resnet50_int8_pretrained_model.pb・・・量子化後のINT8のモデルファイルの出力先
- --callback "python3 ./infer_tf.py --input_graph {} --dataset_dir /imagenet/images --num_images 1000"・・・これが一番重要かつややこしい。量子化処理中にラフに量子化したモデルを一度推論して、FP32で表現されている各数値の使用されているダイナミックレンジをのMinとMaxを求める処理が実施されます。そのための推論スクリプトを実行するためのコマンドを指定します。この際、このスクリプトに入力するモデルファイルのパスはPlaceholderとして{}で指定します。そうすることで、処理中に一時ファイル（ラフな量子化モデル）のパスが動的に代入されます。また、ここでは推論画像枚数としてImagenetのValidation用画像50000枚から1000枚をランダムに使用すようにしていますが、Imagenet以外の画像データを使用する場合や枚数を変えたい場合は適宜変更ください。
- --inputs 'input'・・・元のモデルの入力Opの名称
- --outputs 'predict'・・・元のモデルの出力Opの名称
- --per_channel False・・・
- --excluded_ops ''・・・量子化の対象外とするOps一覧
- --excluded_nodes ''・・・量子化対象外とするNode一覧

おまけとして、[Intel Model Zoo](https://github.com/IntelAI/models)のモデルを使用する場合の方法もご紹介します。

In [ ]:
!python3 api/examples/quantize_model_zoo.py --model resnet50 --in_graph /root/quantization/api/models/resnet50/resnet50_fp32_pretrained_model.pb --out_graph /root/quantization/api/models/resnet50/resnet50_int8_pretrained_model.pb --data_location /imagenet/tfrecord --models_zoo_location /root/models

# 7.量子化後のTensorFlowモデルを実行して性能比較

それでは、量子化後のTensorFlowモデルを実行します。先ほどと同じ推論スクリプトを使用します。

Intel TensorFlowをご使用いただいていれば、アプリケーションコードを変更しなくても、INT8のモデルを自動検知し、適切なCPU命令セット（Intel VNNI等）を実行します。推論処理のスピードがどの程度向上したかをご確認下さい。

In [ ]:
!numactl -l python3 infer_script.py --input_graph "api/models/resnet50/resnet50_int8_pretrained_model.pb" --dataset_dir /imagenet/images  --num_images 50

TensorFlowでの作業は以上となります。

# 8.OpenVINOのIRに変換

ここからはIntel® OpenVINO™ Toolkitを用いた量子化方法をご紹介します。

といってもまずは、元のTensorFlowのモデル（FP32）をOpenVINOのIR（Intermidiate Repretation）形式に変換するところから実施しましょう。

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --input_model=./api/models/resnet50/resnet50_fp32_pretrained_model.pb --input_shape=[1,224,224,3]

念のため、IR(xml+bin)が生成されていることを確認します。

In [ ]:
!ls -la

更に、IRをOpenVINOの推論エンジン（IE）上で実行してみます。TensorFlowの時と同じ推論スクリプトを使用します。モデルはFP32のままですが、IRに変換することでモデルの内部構造がCPUに最適化され、大きく性能が向上したことが確認できるかと思います。

In [ ]:
!numactl -l python3 infer_script.py --input_graph "resnet50_fp32_pretrained_model.xml" --dataset_dir /imagenet/images --num_images 50 --openvino

# 9.IRの量子化実行
IRの量子化はOpenVINOのPOT（Post-Training Optimization Toolkit）を使用して行います。事前にPOTの[セットアップ](https://docs.openvinotoolkit.org/latest/_README.html#install_post_training_optimization_toolkit)を完了させて下さい。

その後、量子化のための各種設定を記述したConfigファイル（JSON）を準備（ダウンロード）します。

In [ ]:
!wget https://raw.githubusercontent.com/hiouchiy/IntelAI/master/tensorflow_quantization/resnet50_int8.json

次に今回使用するConfigファイルの中身を見てみましょう。

In [ ]:
!cat resnet50_int8.json

続いて、POTを使って量子化を実行します。

In [ ]:
!pot -c resnet50_int8.json

実行が成功すると、resultsというフォルダが作成されます。そして、量子化済みのIRがその中に格納されています。

results/se_resnet50_DefaultQuantization/日付日時のフォルダ/optimized/**.xml

ちなみに、POTコマンドではなく、[Pythonスクリプト](https://docs.openvinotoolkit.org/latest/_sample_README.html#how_to_run_the_sample)を書いて同様のことを実現可能することも可能です。より細かなカスタマイズを行いたい時などはぜひご利用ください

# 10.量子化後のIRを実行

In [ ]:
!numactl -l python3 infer_script.py --input_graph "results/se_resnet50_DefaultQuantization/2020-06-07_11-15-38/optimized/se_resnet50.xml" --dataset_dir /imagenet/images --num_images 50 --openvino

おまけ。AccuracyChekerを使用したモデルの精度の確認方法。

In [ ]:
!pot -c resnet50_int8.json -e -d

# 11.それぞれの結果をグラフ化して比較

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

w = 0.4

Y1 = [tf_total_time - tf_infer_time, cpu_total_time - cpu_infer_time]
Y2 = [tf_infer_time, cpu_infer_time]

X = np.arange(len(Y1))

plt.bar(X, Y1, color='gray', width=w, label='Pre/Post', align="center")
plt.bar(X, Y2, color='blue', width=w, bottom=Y1, label='Inference', align="center")

plt.legend(loc="best")
plt.title('Custom Visoin Model Performance Comparison')
plt.ylabel("Spent time per one image (msec)")

plt.xticks(X, ['TensorFlow(CPU)','OpenVINO(CPU)'])

plt.show()

ちなみに公式ベンチマークツールでの測定方法はこちら↓

In [ ]:
!pip install progress

In [ ]:
!numactl -l python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py -m resnet50_fp32_pretrained_model.xml -i ILSVRC2012_val_00000001.JPEG -niter 100 -b 1 -nireq 1 -nstreams 1

In [ ]:
!cp /opt/intel/openvino/deployment_tools/tools/post_training_optimization_toolkit/configs/examples/quantization/classification/se_resnet50_pytorch_int8.json .

In [ ]:
!numactl -N 0 -m 0 python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py -m results/se_resnet50_DefaultQuantization/2020-06-07_06-52-09/optimized/se_resnet50.xml -i ILSVRC2012_val_00000001.JPEG -niter 100 -b 1 -nireq 1 -nstreams 1

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --input_model=model.pb --input_shape=[1,224,224,3]

In [ ]:
!numactl -N 0 -m 0 python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py -m model.xml -i ILSVRC2012_val_00000001.JPEG -niter 100 -b 1 -nireq 1 -nstreams 1

In [ ]:
!pot -c se_azure_cv_int8.json

In [ ]:
!numactl -N 0 -m 0 python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py -m results/se_resnet50_DefaultQuantization/2020-06-07_07-25-25/optimized/se_resnet50.xml -i ILSVRC2012_val_00000001.JPEG -niter 100 -b 1 -nireq 1 -nstreams 1

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time #ここ追加

def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    image = image.convert('RGB')
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

def run_normal(imageFile):
    start1 = time.time() #ここ追加
    
    graph_def = tf.compat.v1.GraphDef()
    labels = []

    # These are set to the default names from exported models, update as needed.
    filename = "api/models/resnet50/resnet50_fp32_pretrained_model.pb"
    labels_filename = "labels.txt"

    # Import the TF graph
    with tf.io.gfile.GFile(filename, 'rb') as f:
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    # Create a list of labels.
#    with open(labels_filename, 'rt') as lf:
#        for l in lf:
#            labels.append(l.strip())

    # Load from a file
    image = Image.open(imageFile)

    # Update orientation based on EXIF tags, if the file has orientation info.
    image = update_orientation(image)

    # Convert to OpenCV format
    image = convert_to_opencv(image)

    # If the image has either w or h greater than 1600 we resize it down respecting
    # aspect ratio such that the largest dimension is 1600
    image = resize_down_to_1600_max_dim(image)

    # We next get the largest center square
    h, w = image.shape[:2]
    min_dim = min(w,h)
    max_square_image = crop_center(image, min_dim, min_dim)

    # Resize that square down to 256x256
    augmented_image = resize_to_256_square(max_square_image)

    # Get the input size of the model
    with tf.compat.v1.Session() as sess:
        input_tensor_shape = sess.graph.get_tensor_by_name('Placeholder:0').shape.as_list()
    network_input_size = input_tensor_shape[1]

    # Crop the center for the specified network_input_Size
    augmented_image = crop_center(augmented_image, network_input_size, network_input_size)

    # These names are part of the model and cannot be changed.
    output_layer = 'predict:0'
    input_node = 'input:0'

    with tf.compat.v1.Session() as sess:
        try:
            prob_tensor = sess.graph.get_tensor_by_name(output_layer)
            start2 = time.time() #ここ追加
            predictions,  = sess.run(prob_tensor, {input_node: [augmented_image] })
            infer_time = time.time() - start2
        except KeyError:
            print ("Couldn't find classification output layer: " + output_layer + ".")
            print ("Verify this a model exported from an Object Detection project.")
            exit(-1)

        # Print the highest probability label
        highest_probability_index = np.argmax(predictions)
        total_time = time.time() - start1
        #print(imageFile + ',', 'Total Time: ' + str(int(total_time*1000.0)) + 'msec,', 'Infer Time: ' + str(int(infer_time*1000.0)) + 'msec,', 'Pred Label: ' + labels[highest_probability_index]) #ここ追加
        print(imageFile + ',', 'Total Time: ' + str(int(total_time*1000.0)) + 'msec,', 'Infer Time: ' + str(int(infer_time*1000.0)) + 'msec,') #ここ追加


In [ ]:
#テスト用の画像をランダムに選択して推論実行
import glob
import random

#file_list = glob.glob("test/*/*")
#img_path = random.choice(file_list)
img_path = "ILSVRC2012_val_00000001.JPEG"
run_normal(img_path)

# ここから

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time #ここ追加

class Model(object):

    def __init__(self):
        self.labels = []
        labels_filename = "labels.txt"

        # Create a list of labels.
#        with open(labels_filename, 'rt') as lf:
#            for l in lf:
#                self.labels.append(l.strip())

    def predict(self, imageFile):
        raise NotImplementedError
    
    def convert_to_opencv(self, image):
        # RGB -> BGR conversion is performed as well.
        image = image.convert('RGB')
        r,g,b = np.array(image).T
        opencv_image = np.array([b,g,r]).transpose()
        return opencv_image

    def crop_center(self, img,cropx,cropy):
        h, w = img.shape[:2]
        startx = w//2-(cropx//2)
        starty = h//2-(cropy//2)
        return img[starty:starty+cropy, startx:startx+cropx]

    def resize_down_to_1600_max_dim(self, image):
        h, w = image.shape[:2]
        if (h < 1600 and w < 1600):
            return image

        new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
        return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

    def resize_to_256_square(self, image):
        h, w = image.shape[:2]
        return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

    def update_orientation(self, image):
        exif_orientation_tag = 0x0112
        if hasattr(image, '_getexif'):
            exif = image._getexif()
            if (exif != None and exif_orientation_tag in exif):
                orientation = exif.get(exif_orientation_tag, 1)
                # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
                orientation -= 1
                if orientation >= 4:
                    image = image.transpose(Image.TRANSPOSE)
                if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                    image = image.transpose(Image.FLIP_TOP_BOTTOM)
                if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                    image = image.transpose(Image.FLIP_LEFT_RIGHT)
        return image

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time #ここ追加

class TFModel(Model):

    def __init__(self, modelFilePath):
        super(TFModel, self).__init__()
        
        graph_def = tf.compat.v1.GraphDef()

        # These are set to the default names from exported models, update as needed.
        #filename = "api/models/resnet50/resnet50_fp32_pretrained_model.pb"
        filename = modelFilePath

        # Import the TF graph
        with tf.io.gfile.GFile(filename, 'rb') as f:
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session()
                
    def predict(self, imageFile):
        start1 = time.time() #ここ追加

        # Load from a file
        image = Image.open(imageFile)

        # Update orientation based on EXIF tags, if the file has orientation info.
        image = super().update_orientation(image)

        # Convert to OpenCV format
        image = super().convert_to_opencv(image)

        # If the image has either w or h greater than 1600 we resize it down respecting
        # aspect ratio such that the largest dimension is 1600
        image = super().resize_down_to_1600_max_dim(image)

        # We next get the largest center square
        h, w = image.shape[:2]
        min_dim = min(w,h)
        max_square_image = super().crop_center(image, min_dim, min_dim)

        # Resize that square down to 256x256
        augmented_image = super().resize_to_256_square(max_square_image)

        # Get the input size of the model
        input_tensor_shape = self.sess.graph.get_tensor_by_name('input:0').shape.as_list()
        network_input_size = input_tensor_shape[1]

        # Crop the center for the specified network_input_Size
        augmented_image = super().crop_center(augmented_image, network_input_size, network_input_size)
        frame = augmented_image

        # These names are part of the model and cannot be changed.
        output_layer = 'predict:0'
        input_node = 'input:0'

        try:
            prob_tensor = self.sess.graph.get_tensor_by_name(output_layer)
            start2 = time.time() #ここ追加
            predictions, = self.sess.run(prob_tensor, {input_node: [augmented_image] })
            infer_time = time.time() - start2
        except KeyError:
            print ("Couldn't find classification output layer: " + output_layer + ".")
            print ("Verify this a model exported from an Object Detection project.")
            exit(-1)

        # Print the highest probability label
        highest_probability_index = np.argmax(predictions)
        total_time = time.time() - start1
        
        #return total_time, infer_time, self.labels[highest_probability_index], frame  #ここ追加
        return total_time, infer_time, "", frame  #ここ追加


In [ ]:
import glob
import random
import numpy as np
import glob
import random
import time
import os
import cv2
import pandas as pd
from PIL import Image
import PIL
import io
import IPython.display
from IPython.display import clear_output

def run_inference(model_type, target_device='CPU', total=500):
    if model_type == 'tf':
        model = TFModel("api/models/resnet50/resnet50_fp32_pretrained_model.pb")
    elif model_type == 'tf_int8':
        model = TFModel("api/models/resnet50/resnet50_int8_pretrained_model.pb")
    else:
        if target_device == 'GPU':
            model = OpenVINOModel('GPU')
        elif target_device == 'MYRIAD':
            model = OpenVINOModel('MYRIAD')
        else:
            model = OpenVINOModel('CPU')

    total_infer_spent_time = 0
    total_spent_time = 0
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    for i in range(total):
        #file_list = glob.glob("dataset/*")
        #img_path = random.choice(file_list)
        img_path = "ILSVRC2012_val_00000001.JPEG"
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        total_time, infer_time, pred_label, frame = model.predict(img_path)

        total_infer_spent_time += infer_time
        total_spent_time += total_time

        #print(img_path, str(int(total_time*1000.0)) + 'msec', str(int(infer_time*1000.0)) + 'msec', pred_label) #ここ追加
        clear_output(wait=True)
        cv2.putText(frame, str(i) + ':', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame, str(i) + ':', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        cv2.putText(frame, str(img_cat), (10,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame, str(img_cat), (10,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        cv2.putText(frame, str(pred_label), (10,130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame, str(pred_label), (10,130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

        f = io.BytesIO()
        PIL.Image.fromarray(frame).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))

        tmp_se = pd.Series( [img_cat, pred_label, str(int(total_time * 1000)), str(int(infer_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True ) 

    clear_output(wait=True)
    print()
    print('全' + str(total) + '枚 完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / total)*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / total)*1000.0)) + " ms/枚")
    display(list_df)

    return int((total_spent_time / total)*1000.0), int((total_infer_spent_time / total)*1000.0)

In [ ]:
tf_total_time, tf_infer_time = run_inference('tf_int8', total=50)

In [ ]:
!python3 infer_tf.py --input_graph "api/models/resnet50/resnet50_int8_pretrained_model.pb"